In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import classification_report
import re
from collections import Counter

In [2]:
data_df = pd.read_csv('data.csv')
data_df.head()

,url,date,title,body_text,labels
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,"['ข่าว', 'สิ่งแวดล้อม']"
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,"['ข่าว', 'สิ่งแวดล้อม']"
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,"['ข่าว', 'สิ่งแวดล้อม']"
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล..."
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,"['ข่าว', 'สิ่งแวดล้อม']"


In [3]:
data_df['combined_text'] = data_df['title'] + ' ' + data_df['body_text']
data_df = data_df.drop(columns=['url', 'date', 'title', 'body_text'])
data_df.head()

,labels,combined_text
0,"['ข่าว', 'สิ่งแวดล้อม']","นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ"" ประชาไท..."
1,"['ข่าว', 'สิ่งแวดล้อม']",ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ ประชาไท- 23 ส.ค...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย นโยบายที่อนุญาตใ...
3,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล...",ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง ประชาไท-23 ส.ค....


In [4]:
data_df.shape

(67889, 2)

In [5]:
# กำหนดค่าต่ำสุดสำหรับจำนวน labels
min_label_count = 10

# ตรวจสอบจำนวน labels ที่ปรากฏในข้อมูล
label_counts = data_df['labels'].value_counts()
print("จำนวนแต่ละ label ก่อนการกรอง:")
print(label_counts)

# กำหนด labels ที่จะเก็บ โดยไม่รวม labels ที่มีจำนวนตัวอย่างน้อยกว่าค่าที่กำหนด
labels_to_keep = label_counts[label_counts >= min_label_count].index.tolist()
print("Labels ที่จะเก็บ:")
print(labels_to_keep)

# กรองข้อมูลเพื่อลบ labels ที่ไม่มีใน labels_to_keep และลบ NaN และ labels ที่เป็น [] 
filtered_data = data_df[
    data_df['labels'].isin(labels_to_keep) &
    data_df['labels'].notna() & 
    (data_df['labels'].astype(str) != '[]')  # ลบ labels ที่เป็น []
]

# แสดงขนาดของ filtered_data
print(f"ขนาดของ filtered_data: {filtered_data.shape}")

จำนวนแต่ละ label ก่อนการกรอง:
labels
['ข่าว', 'การเมือง']                                                                                                                                                                                                                                        14110
['ข่าว', 'สิ่งแวดล้อม']                                                                                                                                                                                                                                      4180
['บทความ']                                                                                                                                                                                                                                                   1494
['ข่าว', 'ขีดเส้นใต้']                                                                                                                                                                       

In [6]:
filtered_data.head()

,labels,combined_text
0,"['ข่าว', 'สิ่งแวดล้อม']","นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ"" ประชาไท..."
1,"['ข่าว', 'สิ่งแวดล้อม']",ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ ประชาไท- 23 ส.ค...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย นโยบายที่อนุญาตใ...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง ประชาไท-23 ส.ค....
5,"['ข่าว', 'สิ่งแวดล้อม']",สำรวจเส้นทางอดีตถนนสายความมั่งคง คลองลาน - อุ้...


Tokenized by PythaiNLP

In [7]:
filtered_data['tokenized_text'] = filtered_data['combined_text'].apply(lambda x: ' '.join(word_tokenize(x, engine='newmm')))

/var/folders/l7/t9wyr70x2xn4_r156p46wg880000gn/T/ipykernel_52718/3974605809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['tokenized_text'] = filtered_data['combined_text'].apply(lambda x: ' '.join(word_tokenize(x, engine='newmm')))


In [8]:
filtered_data.head()

,labels,combined_text,tokenized_text
0,"['ข่าว', 'สิ่งแวดล้อม']","นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ"" ประชาไท...","นักวิจัย หนุน "" แม้ว "" เปิด "" จีเอ็มโ..."
1,"['ข่าว', 'สิ่งแวดล้อม']",ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ ประชาไท- 23 ส.ค...,ภาค ประชาชน ต้าน เปิด เสรี จีเอ็มโอ ประชา ไท...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย นโยบายที่อนุญาตใ...,จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย ท...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง ประชาไท-23 ส.ค....,กอ ต. เสนอ เลิก ถนน คลอง ลาน - อุ้มผาง ประชา...
5,"['ข่าว', 'สิ่งแวดล้อม']",สำรวจเส้นทางอดีตถนนสายความมั่งคง คลองลาน - อุ้...,สำรวจ เส้นทาง อดีต ถนน สาย ความ มั่ง คง คลอง...


In [9]:
import re
from pythainlp.corpus import thai_stopwords

# กำหนด stop words ภาษาไทย
stopwords = list(thai_stopwords())

# ฟังก์ชันลบสัญลักษณ์พิเศษ, ตัวเลข, และ stop words
def clean_text(text):
    # ลบสัญลักษณ์พิเศษ, ตัวเลข, และเครื่องหมาย ':' ออกจากข้อความ
    text = re.sub(r'[\"\'\(\)\-\.!/,0-9:]', '', text)
    
    # ลบ stop words
    return ' '.join([word for word in text.split() if word not in stopwords])

# ตัวอย่างการใช้งาน
filtered_data['cleaned_text'] = filtered_data['tokenized_text'].apply(clean_text)


/var/folders/l7/t9wyr70x2xn4_r156p46wg880000gn/T/ipykernel_52718/348169435.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['cleaned_text'] = filtered_data['tokenized_text'].apply(clean_text)


In [10]:
filtered_data.head()

,labels,combined_text,tokenized_text,cleaned_text
0,"['ข่าว', 'สิ่งแวดล้อม']","นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ"" ประชาไท...","นักวิจัย หนุน "" แม้ว "" เปิด "" จีเอ็มโ...",นักวิจัย หนุน แม้ว จีเอ็มโอ ประชา ไท สค นักวิจ...
1,"['ข่าว', 'สิ่งแวดล้อม']",ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ ประชาไท- 23 ส.ค...,ภาค ประชาชน ต้าน เปิด เสรี จีเอ็มโอ ประชา ไท...,ประชาชน ต้าน เสรี จีเอ็มโอ ประชา ไท สค นักวิชา...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย นโยบายที่อนุญาตใ...,จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย ท...,จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย อนุญาต...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง ประชาไท-23 ส.ค....,กอ ต. เสนอ เลิก ถนน คลอง ลาน - อุ้มผาง ประชา...,กอ ต เสนอ เลิก ถนน คลอง ลาน อุ้มผาง ประชา ไท ส...
5,"['ข่าว', 'สิ่งแวดล้อม']",สำรวจเส้นทางอดีตถนนสายความมั่งคง คลองลาน - อุ้...,สำรวจ เส้นทาง อดีต ถนน สาย ความ มั่ง คง คลอง...,สำรวจ เส้นทาง ถนน สาย มั่ง คลอง ลาน อุ้มผาง ภา...


In [11]:
filtered_data = filtered_data.drop(columns=['combined_text', 'tokenized_text'])

In [12]:
filtered_data.head()

,labels,cleaned_text
0,"['ข่าว', 'สิ่งแวดล้อม']",นักวิจัย หนุน แม้ว จีเอ็มโอ ประชา ไท สค นักวิจ...
1,"['ข่าว', 'สิ่งแวดล้อม']",ประชาชน ต้าน เสรี จีเอ็มโอ ประชา ไท สค นักวิชา...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย อนุญาต...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอ ต เสนอ เลิก ถนน คลอง ลาน อุ้มผาง ประชา ไท ส...
5,"['ข่าว', 'สิ่งแวดล้อม']",สำรวจ เส้นทาง ถนน สาย มั่ง คลอง ลาน อุ้มผาง ภา...


In [13]:
# ตรวจสอบว่ามีค่าที่เป็น NaN ใน DataFrame หรือไม่
nan_rows = filtered_data[filtered_data.isna().any(axis=1)]

# แสดงจำนวนแถวที่มี NaN และตัวอย่างของแถวเหล่านั้น
nan_count = nan_rows.shape[0]
print(f"จำนวนแถวที่มี NaN: {nan_count}")
print("ตัวอย่างแถวที่มี NaN:")
print(nan_rows)

จำนวนแถวที่มี NaN: 0
ตัวอย่างแถวที่มี NaN:
Empty DataFrame
Columns: [labels, cleaned_text]
Index: []


In [14]:
# แสดงสถิติของคอลัมน์ทั้งหมด
print(filtered_data.describe(include='all'))

                      labels  \
count                  22051   
unique                    35   
top     ['ข่าว', 'การเมือง']   
freq                   14110   

                                             cleaned_text  
count                                               22051  
unique                                              21776  
top     วอ ล เดน เบล โล WTO ฮ่องกง แย่มาก สำหรับ อ่าน ...  
freq                                                    3  


In [15]:
from collections import Counter

# นำข้อความที่ผ่านการลบ stopwords มารวมกันทั้งหมดแล้วตัดคำแยกออก
all_words = ' '.join(filtered_data['cleaned_text']).split()

# นับจำนวนคำที่พบ
word_counts = Counter(all_words)

# จัดเรียงคำตามจำนวนการปรากฏมากไปน้อย
sorted_word_counts = word_counts.most_common(40)

# แสดง 20 คำที่ปรากฏบ่อยที่สุด
for word, count in sorted_word_counts:
    print(f"'{word}': {count} ครั้ง")

'คน': 128973 ครั้ง
'ไทย': 109428 ครั้ง
'ประชาชน': 79654 ครั้ง
'เรื่อง': 75662 ครั้ง
'ปี': 73078 ครั้ง
'ประเทศ': 69102 ครั้ง
'รัฐบาล': 67721 ครั้ง
'ทำ': 55986 ครั้ง
'ที่จะ': 51064 ครั้ง
'พื้นที่': 50953 ครั้ง
'แรงงาน': 48735 ครั้ง
'วันที่': 45712 ครั้ง
'รัฐ': 42797 ครั้ง
'จังหวัด': 42296 ครั้ง
'สังคม': 39533 ครั้ง
'ผม': 38643 ครั้ง
'สร้าง': 37593 ครั้ง
'สิทธิ': 34702 ครั้ง
'ปัญหา': 33892 ครั้ง
'บาท': 33463 ครั้ง
'กล่าวว่า': 31959 ครั้ง
'กรณี': 29937 ครั้ง
'ชาวบ้าน': 29487 ครั้ง
'รัฐธรรมนูญ': 28660 ครั้ง
'จำนวน': 28418 ครั้ง
'ระบบ': 28326 ครั้ง
'กฎหมาย': 28052 ครั้ง
'ที่ผ่านมา': 27951 ครั้ง
'ล้าน': 27705 ครั้ง
'บริษัท': 27523 ครั้ง
'อ': 26867 ครั้ง
'เกิดขึ้น': 26299 ครั้ง
'คณะกรรมการ': 26197 ครั้ง
'โครงการ': 26162 ครั้ง
'ชุมนุม': 26093 ครั้ง
'การเมือง': 26066 ครั้ง
'ชุมชน': 25565 ครั้ง
'ดี': 25414 ครั้ง
'ประชาธิปไตย': 24536 ครั้ง
'เดือน': 24371 ครั้ง


In [16]:
filtered_data.head(11)

,labels,cleaned_text
0,"['ข่าว', 'สิ่งแวดล้อม']",นักวิจัย หนุน แม้ว จีเอ็มโอ ประชา ไท สค นักวิจ...
1,"['ข่าว', 'สิ่งแวดล้อม']",ประชาชน ต้าน เสรี จีเอ็มโอ ประชา ไท สค นักวิชา...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย อนุญาต...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอ ต เสนอ เลิก ถนน คลอง ลาน อุ้มผาง ประชา ไท ส...
5,"['ข่าว', 'สิ่งแวดล้อม']",สำรวจ เส้นทาง ถนน สาย มั่ง คลอง ลาน อุ้มผาง ภา...
7,"['ข่าว', 'สิ่งแวดล้อม']",ปี เส้นทาง สาย โครงการ ถนน สาย คลอง ลาน อุ้มผา...
8,"['ข่าว', 'สิ่งแวดล้อม']",เบนซิน ขยับ บาท ลิตร กรุงเทพฯ สค กระทรวง พลังง...
9,['บทความ'],บทวิเคราะห์ ความสัมพันธ์ บรรษัทข้ามชาติ สื่อ ร...
10,"['ข่าว', 'สิ่งแวดล้อม']",จับตา วุฒิ สรรหา กทช กรุงเทพฯ สค ประธาน วุฒิสภ...
11,"['ข่าว', 'สิ่งแวดล้อม']",รัฐ ประกาศ แก้ หนี้ ระบบ สำเร็จ กรุงเทพฯ สค พล...


Tokenizer

In [19]:
# ตรวจสอบจำนวนแต่ละ label
label_counts = data_df['labels'].value_counts()
print("จำนวนแต่ละ label ก่อนการกรอง:")
print(label_counts)


จำนวนแต่ละ label ก่อนการกรอง:
labels
['ข่าว', 'การเมือง']                                                                                                                                                                                                                                        14110
['ข่าว', 'สิ่งแวดล้อม']                                                                                                                                                                                                                                      4180
['บทความ']                                                                                                                                                                                                                                                   1494
['ข่าว', 'ขีดเส้นใต้']                                                                                                                                                                       

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(filtered_data['labels'])
print("ตัวอย่าง label ที่ถูกเข้ารหัส:")
print(labels_encoded[:5])  # แสดงตัวอย่าง 5 label ที่ถูกเข้ารหัส

ตัวอย่าง label ที่ถูกเข้ารหัส:
[[1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]]


In [23]:
from keras.preprocessing.text import Tokenizer


tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(filtered_data['cleaned_text'])
max_len = 500  # ความยาวสูงสุดของลำดับ
padded_sequences = pad_sequences(sequences, maxlen=max_len)

print("ตัวอย่างลำดับข้อความ:")
print(padded_sequences[:5])  # แสดงตัวอย่าง 5 ลำดับ


ตัวอย่างลำดับข้อความ:
[[   0    0    0 ... 1053   87  151]
 [   0    0    0 ... 1053   87  151]
 [   0    0    0 ... 1053   87  151]
 [   0    0    0 ... 1053   87  151]
 [   6  220 2782 ... 1053   87  151]]


In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 500
data = pad_sequences(sequences, maxlen=max_len)
print("ขนาดของข้อมูลหลังทำ padding:")
print(data.shape)  # แสดงขนาดของข้อมูลหลังทำ padding


ขนาดของข้อมูลหลังทำ padding:
(22051, 500)


In [19]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(filtered_data['labels'])
print("ตัวอย่าง label ที่ถูกเข้ารหัส:")
print(labels_encoded[:5])  # แสดงตัวอย่าง 5 label ที่ถูกเข้ารหัส


ตัวอย่าง label ที่ถูกเข้ารหัส:
[[1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]]


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels_encoded, test_size=0.2, random_state=42)
print(f"ขนาดของชุดฝึก: {X_train.shape}, ขนาดของชุดทดสอบ: {X_test.shape}")


ขนาดของชุดฝึก: (17640, 500), ขนาดของชุดทดสอบ: (4411, 500)


In [26]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_len))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(mlb.classes_), activation='sigmoid'))  # sigmoid สำหรับ multi-label classification

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5


2024-10-08 23:08:52.110703: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


441/441 [==============================] - 15s 34ms/step - loss: 0.1640 - accuracy: 0.0000e+00 - val_loss: 0.1282 - val_accuracy: 0.0000e+00
Epoch 2/5
441/441 [==============================] - 14s 33ms/step - loss: 0.0888 - accuracy: 0.0089 - val_loss: 0.1223 - val_accuracy: 0.0136
Epoch 3/5
441/441 [==============================] - 15s 35ms/step - loss: 0.0416 - accuracy: 0.0094 - val_loss: 0.1537 - val_accuracy: 0.0215
Epoch 4/5
441/441 [==============================] - 17s 39ms/step - loss: 0.0265 - accuracy: 0.0223 - val_loss: 0.1685 - val_accuracy: 0.0445
Epoch 5/5
441/441 [==============================] - 15s 34ms/step - loss: 0.0199 - accuracy: 0.0330 - val_loss: 0.1900 - val_accuracy: 0.0349


In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [27]:
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # แปลงค่าความน่าจะเป็นเป็น 0 หรือ 1

138/138 [==============================] - 1s 7ms/step


In [29]:
from sklearn.metrics import classification_report, accuracy_score

macro_accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Macro Accuracy: {macro_accuracy}')
print(classification_report(y_test, y_pred_classes, target_names=mlb.classes_))


Macro Accuracy: 0.6769440036272953
              precision    recall  f1-score   support

                   0.94      0.97      0.95      4034
           '       1.00      1.00      1.00      4411
           ,       0.94      0.96      0.95      4034
           -       0.00      0.00      0.00         7
           .       0.00      0.00      0.00         8
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         7
           [       1.00      1.00      1.00      4411
           ]       1.00      1.00      1.00      4411
           ก       0.84      0.93      0.88      3026
           ข       0.93      0.98      0.95      4051
           ค       0.56      0.27      0.37       463
           ง       0.92      0.95      0.93      3918
           จ       0.80      0.67      0.73       109
           ช       0.79      0.58      0.67       133
           ญ       0.00      0.00      0.00   

/opt/anaconda3/envs/nlp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_new_text(new_text):
    tokenized_text = ' '.join(word_tokenize(new_text, engine='newmm'))
    cleaned_text = clean_text(tokenized_text)  # ใช้ฟังก์ชันทำความสะอาด
    sequence = tokenizer.texts_to_sequences([cleaned_text])  # แปลงเป็นลำดับตัวเลข
    padded_sequence = pad_sequences(sequence, maxlen=max_len)  # ทำ padding
    prediction = model.predict(padded_sequence)  # ทำนาย
    predicted_classes = (prediction > 0.5).astype(int)  # เปลี่ยนค่าความน่าจะเป็นเป็น 0 หรือ 1
    return mlb.inverse_transform(predicted_classes)  # คืนค่าชื่อ label

# ตัวอย่างการใช้งาน
new_text = "ประยุทธ์"
print(predict_new_text(new_text))